In [1]:
# LeNet Class

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import optimizers
from chainer import Variable
from chainer import cuda

import numpy as np

class LeNet(chainer.Chain):
    def __init__(self, num_class, train=True):
        super(LeNet, self).__init__()
        with self.init_scope():
            self.conv1=L.Convolution2D(None, 6, 5, stride=1)
            self.conv2=L.Convolution2D(None, 16, 5, stride=1)
            self.conv3=L.Convolution2D(None, 120, 5, stride=1)
            self.fc4=L.Linear(None, 84)
            self.fc5=L.Linear(None, num_class)
            
    def __call__(self, x):
        h1 = F.max_pooling_2d(F.local_response_normalization(
            F.sigmoid(self.conv1(x))), 2, stride=2)
        h2 = F.max_pooling_2d(F.local_response_normalization(
            F.sigmoid(self.conv2(h1))), 2, stride=2)
        h3 = F.sigmoid(self.conv3(h2))
        h4 = F.sigmoid(self.fc4(h3))
        h5 = self.fc5(h4)
        
        return h5

In [2]:
# MNIST data set

train, test = chainer.datasets.get_mnist()
_xs, ts = train._datasets
_txs, tts = test._datasets

size = 10000
_xs = _xs[:size]
ts = ts[:size]
_txs = _txs[:size]
_txs = _txs[:size]

In [3]:
# padding (60000, 784) -> (60000, 1, 28, 28) -> (60000, 1, 32, 32)

_v0 = np.row_stack((np.zeros(28), np.zeros(28)))
v0 = np.array(_v0)
_h0 = np.column_stack((np.zeros(32), np.zeros(32)))
h0 = np.array(_h0)

def padding(x):
    tmp1 = np.vstack((x, v0))
    tmp2 = np.vstack((v0, tmp1))
    _tmp1 = np.hstack((tmp2, h0))
    _tmp2 = np.hstack((h0, _tmp1))
    return _tmp2

xs_list = []
for i in range(len(_xs)):
    x = np.reshape(_xs[i], (28, 28))
    pad_x = padding(x)
    xs_list.append(pad_x[np.newaxis, :, :])
txs_list = []
for i in range(len(_txs)):
    tx = np.reshape(_txs[i], (28, 28))
    pad_tx = padding(tx)
    txs_list.append(pad_tx[np.newaxis, :, :])
    
xs = np.array(xs_list, dtype=np.float32)
txs = np.array(txs_list, dtype=np.float32)

In [4]:
# method

def check_accuracy(model, xs, ts, batchsize):
    loss = 0
    num_cors = 0
    for i in range(0, len(xs), batchsize):
        x = xs[i:i + batchsize]
        t = ts[i:i + batchsize]
        
        #var_xs = Variable(cuda.to_gpu(x))
        var_xs = Variable(x)
        t = Variable(np.array(t, "i"))
        ys = model(var_xs)
    
        loss += F.softmax_cross_entropy(ys, t)
        ys = np.argmax(ys.data, axis=1)
        #_t = cuda.to_gpu(np.array(cuda.to_cpu(t.data), dtype=np.float32))
        _t = np.array(t.data, dtype=np.float32)
        cors = (ys == _t)
        num_cors += sum(cors)
    accuracy = num_cors / ts.shape[0]
    return accuracy, loss

In [5]:
# learn

model = LeNet(10)
#optimizer = optimizers.SGD()
optimizer = optimizers.Adam()
optimizer.setup(model)

batchsize = 100
datasize = len(xs)

# use GPU
#chainer.cuda.get_device_from_id(0).use()
#model.to_gpu()

#xp = cuda.cupy

for epoch in range(20):
    for i in range(0, datasize, batchsize):
        x = xs[i:i + batchsize]
        t = ts[i:i + batchsize]
        
        #var_x = Variable(cuda.to_gpu(x))
        var_x = Variable(x)
        t = Variable(np.array(t, "i"))
        y = model(var_x)
        
        model.cleargrads()
        loss = F.softmax_cross_entropy(y, t)
        loss.backward()
        optimizer.update()
    accuracy_train, loss_train = check_accuracy(model, xs, ts, batchsize)
    accuracy_test, _           = check_accuracy(model, txs, tts, batchsize)
    
    optimizer.new_epoch()
    
    #print("1: weight={0}, bias={1}".format(model.conv1.W, model.conv1.b))
    print("Epoch {0} loss(train) = {1}, accuracy(train) = {2}, accuracy(test) = {3}".format(epoch + 1, loss_train, accuracy_train, accuracy_test))

Epoch 1 loss(train) = variable(230.29222106933594), accuracy(train) = 0.1032, accuracy(test) = 0.101
Epoch 2 loss(train) = variable(219.98751831054688), accuracy(train) = 0.3816, accuracy(test) = 0.3776
Epoch 3 loss(train) = variable(129.0580291748047), accuracy(train) = 0.6319, accuracy(test) = 0.6383
Epoch 4 loss(train) = variable(82.83393096923828), accuracy(train) = 0.7655, accuracy(test) = 0.7681
Epoch 5 loss(train) = variable(61.799598693847656), accuracy(train) = 0.8188, accuracy(test) = 0.8188
Epoch 6 loss(train) = variable(49.18644332885742), accuracy(train) = 0.8522, accuracy(test) = 0.8515
Epoch 7 loss(train) = variable(41.29401397705078), accuracy(train) = 0.8796, accuracy(test) = 0.8761
Epoch 8 loss(train) = variable(35.63551330566406), accuracy(train) = 0.8989, accuracy(test) = 0.8936
Epoch 9 loss(train) = variable(31.07719612121582), accuracy(train) = 0.9149, accuracy(test) = 0.9097
Epoch 10 loss(train) = variable(27.34433937072754), accuracy(train) = 0.926, accuracy(tes